In [249]:
import pandas as pd
import numpy as np

In [250]:
%ls

AHP-Automation.ipynb          dataset.txt
Corrosion Rate - Vikor.ipynb  user-data.csv
dataset.csv.numbers*          working-dataset.csv


In [251]:
data = pd.read_csv('working-dataset.csv ')

In [252]:
data

,Material,Approx. Cost per Square Meter (USD),Durability (Months),Weight (kg/mÂ²),Corrosion Resistance,Temperature Resistance (Â°C),Manufacturing Complexity,Availability
0,Nickel-Based Superalloys,$750,10 years,300,Very High,1000,Medium,Very High
1,Stainless Steel,$200,8 years,200,High,250,Medium,High
2,Titanium Alloys,$1050,12 years,150,Very High,1100,High,Medium
3,Carbon Fiber Reinforced Polymers (CFRP),$800,5 years,50,Very High,300,Medium,Medium
4,Ceramic Matrix Composites (CMCs),$1350,12 years,50,Very High,1050,High,Low
5,Inconel,$700,15 years,200,Very High,800,Medium,Very High
6,Single Crystal Alloys,$1100,13 years,200,Very High,1100,High,Low


In [253]:
pd.DataFrame(data['Material'])

,Material
0,Nickel-Based Superalloys
1,Stainless Steel
2,Titanium Alloys
3,Carbon Fiber Reinforced Polymers (CFRP)
4,Ceramic Matrix Composites (CMCs)
5,Inconel
6,Single Crystal Alloys


# Mapping Five Scale values

In [254]:
data_mapping = {
    'Very Low':1,
    'Low':2,
    'Medium':3,
    'High':4,
    'Very High':5
}

In [255]:
def columns_map(df,columns):
    df_new = df.copy()
    for col in df[columns]:
        df_new[col] = df_new[col].map(data_mapping)
    return df_new

In [256]:
mapped_data = columns_map(data,['Corrosion Resistance','Manufacturing Complexity','Availability'])
mapped_data.drop(columns = ['Material'],inplace=True)

In [257]:
weight = [0.14, 0.29, 0.04, 0.19, 0.14, 0.1, 0.1]

In [258]:
def data_correction(df):
    df_new = df.copy()
    df_new['Approx. Cost per Square Meter (USD)']=df_new['Approx. Cost per Square Meter (USD)'].str.replace('$','')
    df_new['Durability (Months)']=df_new['Durability (Months)'].str.replace('years','').str.strip()
    df_new['Approx. Cost per Square Meter (USD)'] = df_new['Approx. Cost per Square Meter (USD)'].astype('int')
    df_new['Durability (Months)'] = df_new['Durability (Months)'].astype('float64')
    return df_new

In [259]:
# do when it is needed
mapped_data = data_correction(mapped_data)

In [260]:
Beneficial_criteria = ['Durability (Months)','Corrosion Resistance','Temperature Resistance (Â°C)','Availability']
NonBeneficial_criteria = ['Approx. Cost per Square Meter (USD)','Weight (kg/mÂ²)','Manufacturing Complexity']

Step 1: Normalization

In [261]:
def return_max_minCase(df,beneficial,non_beneficial):
    df_new = df.copy()
    best = []
    worst =[]
    for col in df_new.columns:
        if col in non_beneficial:
            best.append(min(df_new[col]))
            worst.append(max(df_new[col]))
        else:
            best.append(max(df_new[col]))
            worst.append(min(df_new[col])) 
    return best,worst,df_new

In [262]:
Best,Worst,mapped_data = return_max_minCase(mapped_data,Beneficial_criteria,NonBeneficial_criteria)

In [263]:
Best

[200, 15.0, 50, 5, 1100, 3, 5]

In [264]:
Worst

[1350, 5.0, 300, 4, 250, 4, 2]

In [265]:
mapped_data

,Approx. Cost per Square Meter (USD),Durability (Months),Weight (kg/mÂ²),Corrosion Resistance,Temperature Resistance (Â°C),Manufacturing Complexity,Availability
0,750,10.0,300,5,1000,3,5
1,200,8.0,200,4,250,3,4
2,1050,12.0,150,5,1100,4,3
3,800,5.0,50,5,300,3,3
4,1350,12.0,50,5,1050,4,2
5,700,15.0,200,5,800,3,5
6,1100,13.0,200,5,1100,4,2


In [266]:
def get_Normalized(df):
    df_new = df.copy()
    for i in range(mapped_data.shape[1]):
        for j in range(mapped_data.shape[0]):
               df_new.iloc[j,i] = (round(weight[i] *((Best[i] - df_new.iloc[j,i]) / (Best[i] - Worst[i])),4))
    return df_new

In [267]:
norm_df = get_Normalized(mapped_data)

In [268]:
norm_df

,Approx. Cost per Square Meter (USD),Durability (Months),Weight (kg/mÂ²),Corrosion Resistance,Temperature Resistance (Â°C),Manufacturing Complexity,Availability
0,0.0670,0.145,0.040,0.00,0.0165,0.0,0.0000
1,-0.0000,0.203,0.024,0.19,0.1400,0.0,0.0333
2,0.1035,0.087,0.016,0.00,0.0000,0.1,0.0667
3,0.0730,0.290,-0.000,0.00,0.1318,-0.0,0.0667
4,0.1400,0.087,-0.000,0.00,0.0082,0.1,0.1000
5,0.0609,0.000,0.024,0.00,0.0494,-0.0,0.0000
6,0.1096,0.058,0.024,0.00,0.0000,0.1,0.1000


In [269]:
def sum_of_rows(df):
    df_new = df.copy()
    sum_ = df_new.sum(axis=1)
    df_new['Si'] = sum_
    return sum_,df_new

In [270]:
_,norm_df_new = sum_of_rows(norm_df)

In [271]:
norm_df_new

,Approx. Cost per Square Meter (USD),Durability (Months),Weight (kg/mÂ²),Corrosion Resistance,Temperature Resistance (Â°C),Manufacturing Complexity,Availability,Si
0,0.0670,0.145,0.040,0.00,0.0165,0.0,0.0000,0.2685
1,-0.0000,0.203,0.024,0.19,0.1400,0.0,0.0333,0.5903
2,0.1035,0.087,0.016,0.00,0.0000,0.1,0.0667,0.3732
3,0.0730,0.290,-0.000,0.00,0.1318,-0.0,0.0667,0.5615
4,0.1400,0.087,-0.000,0.00,0.0082,0.1,0.1000,0.4352
5,0.0609,0.000,0.024,0.00,0.0494,-0.0,0.0000,0.1343
6,0.1096,0.058,0.024,0.00,0.0000,0.1,0.1000,0.3916


In [272]:
def find_max(df):
    df_new = df.copy()
    max_ = df_new.apply(max,axis=1)
    df_new['Ri'] = max_
    return max_,df_new

In [273]:
_,norm_df = find_max(norm_df)

In [274]:
norm_df['Si'] = norm_df_new['Si']
norm_df

,Approx. Cost per Square Meter (USD),Durability (Months),Weight (kg/mÂ²),Corrosion Resistance,Temperature Resistance (Â°C),Manufacturing Complexity,Availability,Ri,Si
0,0.0670,0.145,0.040,0.00,0.0165,0.0,0.0000,0.1450,0.2685
1,-0.0000,0.203,0.024,0.19,0.1400,0.0,0.0333,0.2030,0.5903
2,0.1035,0.087,0.016,0.00,0.0000,0.1,0.0667,0.1035,0.3732
3,0.0730,0.290,-0.000,0.00,0.1318,-0.0,0.0667,0.2900,0.5615
4,0.1400,0.087,-0.000,0.00,0.0082,0.1,0.1000,0.1400,0.4352
5,0.0609,0.000,0.024,0.00,0.0494,-0.0,0.0000,0.0609,0.1343
6,0.1096,0.058,0.024,0.00,0.0000,0.1,0.1000,0.1096,0.3916


Step 2 finding, S*, S-, R* and R-

In [275]:
BW_df = norm_df[['Si','Ri']]

In [276]:
Worst,Best,BW_df = return_max_minCase(BW_df,Beneficial_criteria,NonBeneficial_criteria)

In [277]:
BW_df

,Si,Ri
0,0.2685,0.1450
1,0.5903,0.2030
2,0.3732,0.1035
3,0.5615,0.2900
4,0.4352,0.1400
5,0.1343,0.0609
6,0.3916,0.1096


In [278]:
Best,Worst

([0.1343, 0.0609], [0.5903, 0.29])

In [288]:
# Consider V = 0.5
def Q_function(x,Best,Worst,v):
     return (v*((x[0] - Best[0]) / (Worst[0] - Best[0]))+(1-v)*((x[1] - Best[1]) / (Worst[1] - Best[1])))
def return_Qi(df,Best,Worst):
    df_new = df.copy()
    v=0.5
    l = []
    l.append(df_new.apply(lambda x : round(Q_function(x,Best,Worst,v),3),axis=1))
    return l

In [289]:
Qi = return_Qi(BW_df,Best,Worst)

In [290]:
BW_df_new = pd.DataFrame()
BW_df_new['Materials'] = data['Material']
BW_df_new  = pd.concat([BW_df_new,BW_df,pd.DataFrame(Qi).T],axis=1).rename(columns={0:'Qi'})

In [291]:
BW_df_new['Rank'] = BW_df_new['Qi'].rank()

In [292]:
BW_df_new

,Materials,Si,Ri,Qi,Rank
0,Nickel-Based Superalloys,0.2685,0.1450,0.331,2.0
1,Stainless Steel,0.5903,0.2030,0.810,6.0
2,Titanium Alloys,0.3732,0.1035,0.355,3.0
3,Carbon Fiber Reinforced Polymers (CFRP),0.5615,0.2900,0.968,7.0
4,Ceramic Matrix Composites (CMCs),0.4352,0.1400,0.503,5.0
5,Inconel,0.1343,0.0609,0.000,1.0
6,Single Crystal Alloys,0.3916,0.1096,0.388,4.0
